In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!pip install scikit-learn==1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 23.1 MB 1.5 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2


In [3]:
#Data analysis
import pandas as pd
import numpy as np
#Data visualisation
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
sns.set(font_scale=1)
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
#Modeling
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.ensemble import RandomForestClassifier
# from sklearn_crfsuite import CRF, scorers, metrics
# import sklearn_crfsuite
# from sklearn_crfsuite import scorers
# from sklearn_crfsuite import metrics
# from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import classification_report, make_scorer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import scipy.stats

In [4]:
!pip install sklearn_crfsuite
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 965 kB 4.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.1 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=a10146fb6eae1fe1cc2dfe4e40047bcb682b174dd5a1ba3aa8635ab99f6f8966
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [68]:
ROOT = '/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen'

from ast import literal_eval

train_data = pd.read_csv(f"{ROOT}/nested/train_data.csv")
test_data = pd.read_csv(f"{ROOT}/nested/test_data.csv")

train_data['sent'] = train_data['sent'].apply(literal_eval)
train_data['tag'] = train_data['tag'].apply(literal_eval)
train_data['pos'] = train_data['pos'].apply(literal_eval)


test_data['sent'] = test_data['sent'].apply(literal_eval)
test_data['tag'] = test_data['tag'].apply(literal_eval)
test_data['pos'] = test_data['pos'].apply(literal_eval)


In [69]:
train_data = train_data.explode(['sent', 'tag', 'pos'])
test_data = test_data.explode(['sent', 'tag', 'pos'])


In [70]:
test_data

,juz,no_ayat,sent,tag,pos,sentence #
0,2,99,Dan,O+O,CC,106
0,2,99,sesungguhnya,O+O,RB,106
0,2,99,Kami,O+O,PRP,106
0,2,99,telah,O+O,MD,106
0,2,99,menurunkan,O+O,VB,106
...,...,...,...,...,...,...
190,6,120,apa,O+O,WH,909
190,6,120,yang,O+O,SC,909
190,6,120,mereka,O+O,PRP,909
190,6,120,telah,O+O,MD,909


In [71]:
# A class to retrieve the sentences from the dataset
class getsentence(object):
    
    def __init__(self, data):
        self.n_sent = 1.0
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t, z) for w, t, z in zip(s["sent"].values.tolist(),
                                                           s["tag"].values.tolist(),
                                                     s['pos'].values.tolist())]
        self.grouped = self.data.groupby("sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]

In [72]:
getter = getsentence(train_data)

In [73]:
sentences = getter.sentences
#ths is how a sentence will look like.
print(sentences[1])

[('Maha', 'O+O', 'NN'), ('Pemurah', 'O+O', 'NN'), ('lagi', 'O+O', 'RB'), ('Maha', 'O+O', 'NNP'), ('Penyayang', 'O+O', 'NNP')]


In [74]:
from itertools import chain

import nltk
import sklearn
import scipy.stats

import sklearn_crfsuite
from sklearn_crfsuite import scorers,CRF
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn_crfsuite import metrics

In [234]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][2]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        # 'word[-3:]': word[-3:],
        # 'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        # 'postag': postag,
        # 'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][2]
        features.update({
            # '-1:word.lower()': word1.lower(),
            # '-1:word.istitle()': word1.istitle(),
            # '-1:word.isupper()': word1.isupper(),
            # '-1:postag': postag1,
            # '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][2]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            # '+1:postag': postag1,
            # '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


In [235]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label, pos in sent]

In [236]:
#Creating the train and test set
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [237]:
#Creating the CRF model
crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [238]:
try:
    crf.fit(X, y)
except AttributeError:
    pass

In [239]:
test_getter = getsentence(test_data)
test_sentences = test_getter.sentences

In [240]:
print(test_sentences[0])

[('Segala', 'O+O', 'NNP'), ('puji', 'O+O', 'VB'), ('bagi', 'O+O', 'IN'), ('Allah', 'O+O', 'NNP'), ('Tuhan', 'O+O', 'NNP'), ('semesta', 'O+O', 'VB'), ('alam', 'O+O', 'NN')]


In [241]:
#Creating the train and test set
X_test = [sent2features(s) for s in test_sentences]
y_test = [sent2labels(s) for s in test_sentences]

In [242]:
y_pred = crf.predict(X_test)

In [243]:
def get_level_token(seqs, max_lev=2, level=0) : 
  new_all_level = []
  for seq in seqs : 
    new_all_level_i = []
    for tag in seq : 
      new_tag = []
      tag_per_level = tag.split('+')
      for lev in range(max_lev) :
        if lev + 1 > len(tag_per_level) : 
          new_tag.append(tag_per_level[-1])
        else : 
          new_tag.append(tag_per_level[lev])
      new_all_level_i.append(new_tag)
    new_all_level.append(new_all_level_i)
  
  new_seq = []
  for seq in new_all_level :
    seq_i = [] 
    for tag in seq : 
      seq_i.append(tag[level])
    new_seq.append(seq_i)
  
  return new_seq

In [244]:
y_true_0 = get_level_token(y_test, max_lev=2, level=0)
y_pred_0 = get_level_token(y_pred, max_lev=2, level=0)

from seqeval.metrics import classification_report
print(classification_report(y_true_0, y_pred_0, digits=3))

              precision    recall  f1-score   support

         PER      0.737     0.631     0.680       222

   micro avg      0.737     0.631     0.680       222
   macro avg      0.737     0.631     0.680       222
weighted avg      0.737     0.631     0.680       222



In [245]:
y_true_1 = get_level_token(y_test, max_lev=2, level=1)
y_pred_1 = get_level_token(y_pred, max_lev=2, level=1)

from seqeval.metrics import classification_report
print(classification_report(y_true_1, y_pred_1, digits=3))

              precision    recall  f1-score   support

         PER      0.684     0.586     0.631       222

   micro avg      0.684     0.586     0.631       222
   macro avg      0.684     0.586     0.631       222
weighted avg      0.684     0.586     0.631       222



In [220]:
y_test_token = []
for i in y_test : 
  y_test_token = y_test_token + i

y_pred_token = []
for i in y_pred : 
  y_pred_token = y_pred_token + i

In [25]:
from sklearn.metrics import classification_report
print(classification_report(y_test_token, y_pred_token))


              precision    recall  f1-score   support

 B-PER+B-PER       0.96      0.85      0.90       222
 I-PER+I-PER       0.94      0.68      0.79       444
     O+I-PER       0.71      0.13      0.22       114
         O+O       0.96      1.00      0.98      6243

    accuracy                           0.96      7023
   macro avg       0.90      0.66      0.72      7023
weighted avg       0.95      0.96      0.95      7023



In [26]:
test_data['pred'] = y_pred_token

In [27]:
test_data = pd.read_csv(f"{ROOT}/nested/test_data.csv")

test_data['sent'] = test_data['sent'].apply(literal_eval)
test_data['tag'] = test_data['tag'].apply(literal_eval)
test_data['pos'] = test_data['pos'].apply(literal_eval)

test_data = test_data.sort_values(['sentence #'])
test_data['pred'] = y_pred

In [28]:
test_data = test_data.explode(['sent', 'tag', 'pred'])
test_data = test_data[['juz', 'no_ayat', 'sent', 'tag', 'pred']]
test_data

,juz,no_ayat,sent,tag,pred
68,1,2,Segala,O+O,O+O
68,1,2,puji,O+O,O+O
68,1,2,bagi,O+O,O+O
68,1,2,Allah,O+O,O+O
68,1,2,Tuhan,O+O,O+O
...,...,...,...,...,...
124,6,155,bertakwalah,O+O,O+O
124,6,155,agar,O+O,O+O
124,6,155,kamu,O+O,O+O
124,6,155,diberi,O+O,O+O


In [29]:
test_data.to_csv(f"{ROOT}/nested/results-prediction/crf.csv", index=False)

In [30]:
test_data[(test_data.juz == 2) & (test_data.no_ayat == 99)]

,juz,no_ayat,sent,tag,pred
0,2,99,Dan,O+O,O+O
0,2,99,sesungguhnya,O+O,O+O
0,2,99,Kami,O+O,O+O
0,2,99,telah,O+O,O+O
0,2,99,menurunkan,O+O,O+O
0,2,99,kepadamu,O+O,O+O
0,2,99,ayat-ayat,O+O,O+O
0,2,99,yang,O+O,O+O
0,2,99,jelas,O+O,O+O
0,2,99,dan,O+O,O+O


In [ ]:
0.7375
0.741
